In [14]:
'''Projeto: Classificação de Imagens com Ensemble de Modelos

Descrição:
O objetivo do projeto é construir um sistema de classificação de imagens que utiliza modelos treinados em TensorFlow e PyTorch. O ensemble final será integrado e avaliado usando Scikit-learn.

Etapas do Projeto'''

'Projeto: Classificação de Imagens com Ensemble de Modelos\n\nDescrição:\nO objetivo do projeto é construir um sistema de classificação de imagens que utiliza modelos treinados em TensorFlow e PyTorch. O ensemble final será integrado e avaliado usando Scikit-learn.\n\nEtapas do Projeto'

1. Pré-processamento dos Dados (Scikit-learn)


In [15]:


#Usar o Scikit-learn para carregar e dividir o dataset (ex.: CIFAR-10 ou um dataset customizado).
#Normalizar as imagens e aplicar técnicas de aumento de dados, se necessário.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.datasets import cifar10

# Carregar dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalizar os dados
X_train, X_test = X_train / 255.0, X_test / 255.0

# Dividir em treino/validação
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)



2. Modelo com TensorFlow


In [ ]:
#Criar e treinar um modelo de rede neural convolucional (CNN) usando TensorFlow/Keras.
import tensorflow as tf
from tensorflow.keras import layers, models

# Definir o modelo
model_tf = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

# Compilar e treinar
model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_tf.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))



Epoch 1/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 59s 45ms/step - accuracy: 0.3549 - loss: 1.7567 - val_accuracy: 0.5424 - val_loss: 1.2886
Epoch 2/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 78s 42ms/step - accuracy: 0.5768 - loss: 1.1876 - val_accuracy: 0.6115 - val_loss: 1.0904
Epoch 3/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 82s 42ms/step - accuracy: 0.6384 - loss: 1.0352 - val_accuracy: 0.6291 - val_loss: 1.0601
Epoch 4/10
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 79s 40ms/step - accuracy: 0.6706 - loss: 0.9459 - val_accuracy: 0.6350 - val_loss: 1.0382
Epoch 5/10
1131/1250 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.6918 - loss: 0.8761

3. Modelo com PyTorch


In [ ]:
#Treinar um modelo CNN similar usando PyTorch.
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Preparar os dados para PyTorch
train_dataset = TensorDataset(torch.tensor(X_train).permute(0, 3, 1, 2), torch.tensor(y_train).squeeze())
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Definir o modelo
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, activation='relu')
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, activation='relu')
        self.fc1 = nn.Linear(64 * 6 * 6, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 6 * 6)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model_pt = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_pt.parameters(), lr=0.001)

# Treinar o modelo
for epoch in range(10):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model_pt(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()



4. Ensemble com Scikit-learn


In [ ]:
#Combinar as previsões dos dois modelos usando um ensemble (ex.: Random Forest ou Voting Classifier).
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score

# Obter previsões dos modelos
y_pred_tf = model_tf.predict(X_test).argmax(axis=1)
y_pred_pt = model_pt(torch.tensor(X_test).permute(0, 3, 1, 2).float()).detach().numpy

Gráficos

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
import numpy as np
import torch

# 1. Previsões dos modelos
# TensorFlow já está em y_pred_tf
# PyTorch precisa ser transformado com argmax
y_pred_pt_logits = model_pt(torch.tensor(X_test).permute(0, 3, 1, 2).float())
y_pred_pt = y_pred_pt_logits.detach().numpy().argmax(axis=1)

# Ensemble com votação majoritária
y_pred_ensemble = np.array([
    np.bincount([tf, pt]).argmax()
    for tf, pt in zip(y_pred_tf, y_pred_pt)
])

# Corrigir formato de y_test
y_test = y_test.squeeze()

# 2. Matrizes de Confusão
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for ax, y_pred, title in zip(
    axes,
    [y_pred_tf, y_pred_pt, y_pred_ensemble],
    ["TensorFlow", "PyTorch", "Ensemble"]
):
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(cm)
    disp.plot(ax=ax, colorbar=False)
    ax.set_title(f"Matriz de Confusão - {title}")
plt.tight_layout()
plt.show()

# 3. Acurácia Final
acc_tf = accuracy_score(y_test, y_pred_tf)
acc_pt = accuracy_score(y_test, y_pred_pt)
acc_ensemble = accuracy_score(y_test, y_pred_ensemble)

plt.figure(figsize=(6, 5))
plt.bar(["TensorFlow", "PyTorch", "Ensemble"], [acc_tf, acc_pt, acc_ensemble], color=["blue", "orange", "green"])
plt.ylabel("Acurácia")
plt.title("Comparação Final de Acurácia")
plt.ylim(0, 1)
plt.grid(axis="y")
plt.tight_layout()
plt.show()
